# Lesson Learn

- What worked
   - UID 를 찾는 것
     -  Keys to identifying UID : card1, addr1, and D1(days since client (credit card) began).
     -  D1n = TransactionDay -  D1 : 카드 개시일
        - Where TransactionDay = TransactionDT / (24 * 60* 60) ).
     - D3n = TransactionDay - D3 : 마지막 카드 거래일
     - 같은 방법으로 DXn = TransactionDay - DX
     - UID 검증 : UID 로 Groupby 한 데이터들의 D15n 의 표준편차가 0이 아니면 UID 예측은 잘못된 것. 
     - Feature Engineering
     - UID 로 Groupby --> Aggregated Feature를 만드는 것(Group Feature)
   - Frequency Encoding 
   - CatBoost / XGB 등 다양한 모델로 Stacking
   - CV Scheme 은 중요하다
   - Train / Test 가 시간순으로 배열되어 있으므로
   - CV Examples(Time Series)
     - 0 | 2 3 4 5 6
     - 0 1 | 3 4 5 6
     - 0 1 2 | 4 5 6
     - 0 1 2 3 | 5 6
   - Define Validation Set matters
     - Adversarial validation
        - Covariate Shift
          - Feature 의 train 과 test distribution 이 다르다면 test performance 가 불안정할 수 있다
          - 따라서 어떤 Feature 가 train 과 test 에서 다른 분포를 보이는지 확인 필요 
          - 확인하고자 하는 Feature만 이용해 train 에 포함되는지 test 에 포함되는지 확인 위해 lightGBM 으로 AUC 확인해본다
          - 이때 AUC 는 istest = 0 / 1 을 맞추는 binary classification
          - AUC 가 0.5 에 가까울수록 train/test 간 distribution 에 차이는 없다.
          
   

In [ ]:
def covariate_shift(feature):
    df_card1_train = pd.DataFrame(data={feature: train[feature], 'isTest': 0})
    df_card1_test = pd.DataFrame(data={feature: test[feature], 'isTest': 1})

    # Creating a single dataframe
    df = pd.concat([df_card1_train, df_card1_test], ignore_index=True)
    
    # Encoding if feature is categorical
    if str(df[feature].dtype) in ['object', 'category']:
        df[feature] = LabelEncoder().fit_transform(df[feature].astype(str))
    
    # Splitting it to a training and testing set
    X_train, X_test, y_train, y_test = train_test_split(df[feature], df['isTest'], test_size=0.33, random_state=47, stratify=df['isTest'])

    clf = lgb.LGBMClassifier(**params, num_boost_round=500)
    clf.fit(X_train.values.reshape(-1, 1), y_train)
    roc_auc =  roc_auc_score(y_test, clf.predict_proba(X_test.values.reshape(-1, 1))[:, 1])

    del df, X_train, y_train, X_test, y_test
    gc.collect();
    
    return roc_auc